In [43]:
from helperFunctions import *
from constants import *

In [44]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
import numpy

In [45]:
from filterDataScript import *

In [73]:
# filename2 = "part_P002_block_S001"
# XDF_Path2 = "./data/"+filename2+".xdf"
# XDF_Data2 = loadxdf(XDF_Path2)

# filename3 = "part_P003_block_S001"
# XDF_Path3 = "./data/"+filename3+".xdf"
# XDF_Data3 = loadxdf(XDF_Path3)

filename1 = "part_P001_block_S002"
XDF_Path1 = "./data/"+filename1+".xdf"
XDF_Data1 = loadxdf(XDF_Path1)

filename2 = "part_P001_block_S003"
XDF_Path2 = "./data/"+filename1+".xdf"
XDF_Data2 = loadxdf(XDF_Path1)

In [76]:
trial1 = filterDataForPipline(XDF_Data1)
trial2 = filterDataForPipline(XDF_Data2)
# trial2 = filterDataForPipline(XDF_Data2)
# trial3 = filterDataForPipline(XDF_Data3)


In [77]:
model = LinearDiscriminantAnalysis()

In [ ]:
#NO FREQUENCY/POWER BINS:

In [78]:
XTrial1 = []
YTrial1 = []
for i in trial1:
    XTrial1.append(i[1])
    YTrial1.append(i[0])

XTrial2 = []
YTrial2 = []
for i in trial2:
    XTrial2.append(i[1])
    YTrial2.append(i[0])

# XTrial3 = []
# YTrial3 = []
# for i in trial3:
#     XTrial3.append(i[1])
#     YTrial3.append(i[0])

In [79]:
XTrial1Avg = []
for i in range(len(XTrial1)):
    XTrial1Avg.append(np.mean(XTrial1[i], axis = 0))

XTrial2Avg = []
for i in range(len(XTrial2)):
    XTrial2Avg.append(np.mean(XTrial2[i], axis = 0))


# XTrial3Avg = []
# for i in range(len(XTrial3)):
#     XTrial3Avg.append(np.mean(XTrial3[i], axis = 0))

In [83]:
XVarsAvg = XTrial1Avg + XTrial2Avg
yVarsAvg = YTrial1 + YTrial2

In [86]:
XTrain, Xtest, ytrain, ytest = train_test_split(XVarsAvg, yVarsAvg, test_size=0.25, random_state=42)

In [87]:
model.fit(XTrain, ytrain)

/Users/Alvin/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [88]:
YPred= model.predict(Xtest)

In [91]:
from sklearn.metrics import accuracy_score

In [92]:
accuracy_score(ytest, YPred)

0.5

In [ ]:
#POWER BINS

In [93]:
import pyeeg

In [94]:
def getPowerRatio(eeg_data, binning, eeg_fs=250):
    power, power_ratio = pyeeg.bin_power(eeg_data, binning, eeg_fs)
    return np.array(power_ratio)

In [95]:
def getIntervals(binning): 
    intervals = list()
    for i, val in enumerate(binning[:-1]): 
        intervals.append((val, binning[i+1]))
    return intervals

In [96]:
def getEEGDataInMicroVolts(original_data):
    eeg_data = original_data[StreamType.EEG.value][StreamType.DATA.value][:,0]
    return getFilteredEEGDataInMicroVolts_fromEEGList(eeg_data)

In [97]:

def convertRawToMicroVolts(raw_values):
    # http://support.neurosky.com/kb/science/how-to-convert-raw-values-to-voltage 
    # Converts an array of NeuroSky raw values into an array of millivolts 
    return (np.array(np.array(raw_values) * (1.8/4096)) / 2) * 1000


In [98]:
def getFilteredEEGDataInMicroVolts_fromEEGList(eeg_data):
    eeg_data_in_volts = convertRawToMicroVolts(eeg_data)
    sig_filt = filt.filter_signal(eeg_data_in_volts, eeg_fs, 'bandpass', f_range, filter_type='iir', butterworth_order=2)
    return sig_filt

In [99]:
sub_binning=[0.5, 4, 7, 12, 30]
sub_intervals = getIntervals(sub_binning)

In [187]:
trial1Power, _, _ = getMarkerBoundSingleMarkerData('NewImageStart', 'NewImageEnd', XDF_Data1, go_backward=False)
trial2Power, _, _ = getMarkerBoundSingleMarkerData('NewImageStart', 'NewImageEnd', XDF_Data2, go_backward=False)



In [102]:
eeg_fs = getEEGfs(XDF_Data1) 
f_range = (0.5, 50)

In [122]:
face_eeg = []
landscape_eeg = []
fidx = 0
lidx = 0
fdic = {}
ldic = {}
for trial in trial1Power:
    eeg_data_in_volts = getEEGDataInMicroVolts(trial)
    avgs = np.mean(trial.get('eeg').get('data'),axis = 0)
    #print(avgs)
    if 'Face' in trial[StreamType.MARKER.value][StreamType.DATA.value]: 
        fdic[fidx] = avgs
        face_eeg.append(eeg_data_in_volts)
        fidx +=1
    else :
        ldic[lidx] = avgs
        landscape_eeg.append(eeg_data_in_volts)
        lidx += 1

/Users/Alvin/anaconda3/lib/python3.7/site-packages/neurodsp/filt/filter.py:86: UserWarning: Edge artifacts are not removed when using an IIR filter.
  warn('Edge artifacts are not removed when using an IIR filter.')
/Users/Alvin/anaconda3/lib/python3.7/site-packages/neurodsp/filt/iir.py:136: UserWarning: IIR filters are not recommended other than for notch filters.
  warn('IIR filters are not recommended other than for notch filters.')
/Users/Alvin/anaconda3/lib/python3.7/site-packages/neurodsp/filt/utils.py:65: RuntimeWarning: divide by zero encountered in log10
  db = 20 * np.log10(abs(h_vals))


In [189]:
face_eeg2 = []
landscape_eeg2 = []
fidx2 = 0
lidx2 = 0
fdic2 = {}
ldic2 = {}
for trial in trial2Power:
    eeg_data_in_volts = getEEGDataInMicroVolts(trial)
    avgs = np.mean(trial.get('eeg').get('data'),axis = 0)
    #print(avgs)
    if 'Face' in trial[StreamType.MARKER.value][StreamType.DATA.value]: 
        fdic2[fidx2] = avgs
        face_eeg2.append(eeg_data_in_volts)
        fidx2 +=1
    else :
        ldic2[lidx2] = avgs
        landscape_eeg2.append(eeg_data_in_volts)
        lidx2 += 1

/Users/Alvin/anaconda3/lib/python3.7/site-packages/neurodsp/filt/filter.py:86: UserWarning: Edge artifacts are not removed when using an IIR filter.
  warn('Edge artifacts are not removed when using an IIR filter.')
/Users/Alvin/anaconda3/lib/python3.7/site-packages/neurodsp/filt/iir.py:136: UserWarning: IIR filters are not recommended other than for notch filters.
  warn('IIR filters are not recommended other than for notch filters.')
/Users/Alvin/anaconda3/lib/python3.7/site-packages/neurodsp/filt/utils.py:65: RuntimeWarning: divide by zero encountered in log10
  db = 20 * np.log10(abs(h_vals))


In [133]:
power_ratio_landscape = [getPowerRatio(data[:], sub_binning) for data in landscape_eeg]

In [152]:
power_ratio_face = [getPowerRatio(data[:], sub_binning) for data in face_eeg]

In [135]:
power_ratio_landscape = [(0, np.append(power_ratio_landscape[i],ldic[i])) for i in range(len(power_ratio_landscape))]


In [153]:
power_ratio_face = [(1, np.append(power_ratio_face[i],fdic[i])) for i in range(len(power_ratio_face))]

In [190]:
power_ratio_landscape2 = [getPowerRatio(data[:], sub_binning) for data in landscape_eeg2]
power_ratio_landscape2 = [(0, data) for data in power_ratio_landscape2]

power_ratio_face2 = [getPowerRatio(data[:], sub_binning) for data in face_eeg2]
power_ratio_face2 = [(1, data) for data in power_ratio_face2]

In [155]:
power_ratio = power_ratio_face + power_ratio_landscape

In [191]:
power_ratio2 = power_ratio_face2 + power_ratio_landscape2

In [157]:
yVars = []
XVars = []
for i in power_ratio:
    XVars.append(i[1])
    yVars.append(i[0])

In [196]:
yVars2 = []
XVars2 = []
for i in power_ratio2:
    XVars2.append(i[1])
    yVars2.append(i[0])

In [158]:
from sklearn.model_selection import train_test_split

In [197]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(XVars2, yVars2, test_size=0.20, random_state=42)

In [165]:
X_train, X_test, y_train, y_test = train_test_split(XVars, yVars, test_size=0.20, random_state=42)

In [166]:
model.fit(X_train, y_train)

/Users/Alvin/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [167]:
accuracy_score(y_test, model.predict(X_test))

0.7142857142857143

In [198]:
model.fit(X_train2, y_train2)
accuracy_score(y_test2, model.predict(X_test2))

/Users/Alvin/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


0.6964285714285714

In [ ]:
#average is around 70%

In [168]:
##WITHOUT AVGS

In [180]:
power_ratio_landscape = [getPowerRatio(data[:], sub_binning) for data in landscape_eeg]
power_ratio_landscape = [(0, data) for data in power_ratio_landscape]


In [181]:
power_ratio_face = [getPowerRatio(data[:], sub_binning) for data in face_eeg]
power_ratio_face = [(1, data) for data in power_ratio_face]


In [182]:
power_ratio = power_ratio_face + power_ratio_landscape

In [183]:
len(power_ratio)

280

In [184]:
yVars = []
XVars = []
for i in power_ratio:
    XVars.append(i[1])
    yVars.append(i[0])
    
X_train, X_test, y_train, y_test = train_test_split(XVars, yVars, test_size=0.20, random_state=42)

In [185]:
model.fit(X_train, y_train)

/Users/Alvin/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [186]:
accuracy_score(y_test, model.predict(X_test))

0.6964285714285714

In [192]:
#ACROSS TRIALS:


In [193]:
acrossRatios = power_ratio + power_ratio2
yVars = []
XVars = []
for i in power_ratio:
    XVars.append(i[1])
    yVars.append(i[0])
    
X_train, X_test, y_train, y_test = train_test_split(XVars, yVars, test_size=0.20, random_state=42)

In [194]:
model.fit(X_train, y_train)

/Users/Alvin/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [195]:
accuracy_score(y_test, model.predict(X_test))

0.6964285714285714